In [198]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from statsmodels.graphics.tsaplots import plot_acf
import  matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import joblib


In [199]:
df = pd.read_csv("../BTC_1_year_data.csv").set_index("close_time")
df

,open,high,low,close,volume
close_time,,,,,
2024-11-30 15:29:59.999000+00:00,96602.51,96659.50,96532.00,96659.50,133.45453
2024-11-30 15:44:59.999000+00:00,96659.49,96691.69,96602.01,96634.28,117.43398
2024-11-30 15:59:59.999000+00:00,96634.28,96732.15,96602.67,96645.41,85.14427
2024-11-30 16:14:59.999000+00:00,96645.42,96757.06,96615.52,96652.01,106.03529
2024-11-30 16:29:59.999000+00:00,96652.00,96652.01,96542.70,96555.42,241.85001
...,...,...,...,...,...
2025-11-30 14:14:59.999000+00:00,91770.00,91779.94,91641.05,91682.73,57.57016
2025-11-30 14:29:59.999000+00:00,91682.74,91707.30,91483.61,91554.20,117.52310
2025-11-30 14:44:59.999000+00:00,91554.19,91583.61,91256.88,91359.92,236.73655


#### Feature Engineering

In [200]:
df["close_log_return"] = np.log(df["close"]/df["close"].shift())

    Create lagged features

In [201]:
df = df.copy()

In [202]:
def generate_ma_features(df, close_col="close", windows=[5, 10], shift_pct=True):
    """
    Generate moving average and pct-change-to-close features for a list of window sizes.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with at least a 'close' column.
    close_col : str
        Column name for the close price.
    windows : list
        List of integers for rolling windows.
    shift_pct : bool
        Whether to shift pct-change features by 1 step to avoid leakage.

    Returns
    -------
    df : pd.DataFrame
        DataFrame with new features added.
    """
    df = df.copy()

    for w in windows:
        ma_col = f"{w}ma"
        pct_col = f"pct_change_{w}ma_close"

        # Moving average
        df[ma_col] = df[close_col].rolling(w).mean()

        # Percentage difference from MA
        df[pct_col] = (df[close_col] - df[ma_col]) / df[ma_col] * 100

        # Shift to avoid leakage (optional)
        if shift_pct:
            df[pct_col] = df[pct_col].shift()

    return df


windows = [5, 10, 20, 50, 100,200]

df = generate_ma_features(df, windows=windows)


In [203]:
df["close_log_return_lag_1"] = df["close_log_return"].shift()
df["close_log_return_lag_2"] = df["close_log_return"].shift(2)
df["close_log_return_lag_3"] = df["close_log_return"].shift(3)

In [204]:
df = df.dropna(how="any")

    Create binary classification target
        - 1=> Long => Price moves up
        - 0=> Short => Price goes down

In [205]:
df["close_log_return_dir"] = df["close_log_return"].map(lambda x:1 if x>0 else 0)
# df = df.drop(columns="close_log_return")

In [206]:
df

,open,high,low,close,volume,close_log_return,5ma,pct_change_5ma_close,10ma,pct_change_10ma_close,...,50ma,pct_change_50ma_close,100ma,pct_change_100ma_close,200ma,pct_change_200ma_close,close_log_return_lag_1,close_log_return_lag_2,close_log_return_lag_3,close_log_return_dir
close_time,,,,,,,,,,,,,,,,,,,,,
2024-12-02 17:29:59.999000+00:00,96263.99,96324.00,95372.72,95372.73,697.37610,-0.009302,96160.034,-0.284634,96538.085,-0.381805,...,95839.3012,0.415145,96564.7235,-0.330538,96638.00240,-0.393656,0.001003,-0.002374,-0.002114,0
2024-12-02 17:44:59.999000+00:00,95372.73,95498.87,95051.11,95450.00,1061.30282,0.000810,95930.036,-0.818743,96439.485,-1.207145,...,95810.6534,-0.486827,96545.9514,-1.234399,96632.08100,-1.309291,-0.009302,0.001003,-0.002374,1
2024-12-02 17:59:59.999000+00:00,95450.00,95460.00,94395.00,94604.03,954.78259,-0.008902,95571.642,-0.500402,96232.862,-1.026016,...,95768.7340,-0.376423,96520.1119,-1.135160,96621.87410,-1.223280,0.000810,-0.009302,0.001003,0
2024-12-02 18:14:59.999000+00:00,94604.03,95396.00,94488.00,95367.99,846.79767,0.008043,95411.748,-1.012447,96053.661,-1.692594,...,95745.1934,-1.216163,96502.6446,-1.985163,96615.45400,-2.088393,-0.008902,0.000810,-0.009302,1
2024-12-02 18:29:59.999000+00:00,95368.00,95599.00,95235.99,95515.42,315.32601,0.001545,95262.034,-0.045862,95900.403,-0.713842,...,95728.3020,-0.393966,96486.4730,-1.175776,96610.25400,-1.291164,0.008043,-0.008902,0.000810,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-30 14:14:59.999000+00:00,91770.00,91779.94,91641.05,91682.73,57.57016,-0.000951,91692.230,0.131374,91492.959,0.362326,...,91108.4782,0.741975,90944.2066,0.919845,90973.87660,0.876005,0.000886,-0.001771,0.004175,0
2025-11-30 14:29:59.999000+00:00,91682.74,91707.30,91483.61,91554.20,117.52310,-0.001403,91709.364,-0.010361,91543.501,0.207416,...,91121.2960,0.630295,90952.0913,0.812062,90973.85595,0.779183,-0.000951,0.000886,-0.001771,0
2025-11-30 14:44:59.999000+00:00,91554.19,91583.61,91256.88,91359.92,236.73655,-0.002124,91611.110,-0.169191,91555.036,0.011687,...,91129.5986,0.475085,90958.2405,0.662006,90973.25385,0.637924,-0.001403,-0.000951,0.000886,0


In [207]:
def drop_ma_columns(df, windows, extra_drop=None):
    """
    Remove moving-average columns after pct-change features are created.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe.
    windows : list
        List of MA windows used (e.g., [5,10,20]).
    extra_drop : list or None
        Additional columns to drop explicitly.

    Returns
    -------
    df : pd.DataFrame
        DataFrame with MA columns removed.
    """
    df = df.copy()

    # ma columns generated earlier
    ma_cols = [f"{w}ma" for w in windows]

    # combine with user-provided columns
    if extra_drop:
        drop_cols = list(set(ma_cols + extra_drop))
    else:
        drop_cols = ma_cols

    # drop only those that actually exist
    drop_cols = [c for c in drop_cols if c in df.columns]

    df = df.drop(columns=drop_cols, errors="ignore")

    return df

df = drop_ma_columns(df, windows, 
                     extra_drop=["volume", "close_log_return",
                                 "open", "high", "low", "close"])


In [208]:
df.head()

,pct_change_5ma_close,pct_change_10ma_close,pct_change_20ma_close,pct_change_50ma_close,pct_change_100ma_close,pct_change_200ma_close,close_log_return_lag_1,close_log_return_lag_2,close_log_return_lag_3,close_log_return_dir
close_time,,,,,,,,,,
2024-12-02 17:29:59.999000+00:00,-0.284634,-0.381805,0.154271,0.415145,-0.330538,-0.393656,0.001003,-0.002374,-0.002114,0
2024-12-02 17:44:59.999000+00:00,-0.818743,-1.207145,-0.772314,-0.486827,-1.234399,-1.309291,-0.009302,0.001003,-0.002374,1
2024-12-02 17:59:59.999000+00:00,-0.500402,-1.026016,-0.710987,-0.376423,-1.135160,-1.223280,0.000810,-0.009302,0.001003,0
2024-12-02 18:14:59.999000+00:00,-1.012447,-1.692594,-1.572144,-1.216163,-1.985163,-2.088393,-0.008902,0.000810,-0.009302,1
2024-12-02 18:29:59.999000+00:00,-0.045862,-0.713842,-0.775662,-0.393966,-1.175776,-1.291164,0.008043,-0.008902,0.000810,1


    Check class imbalance

In [209]:
print(df["close_log_return_dir"].value_counts())

close_log_return_dir
0    17459
1    17381
Name: count, dtype: int64


#### Split data into training and testing set but in temporal order

    Feature-Target Split

In [210]:
X = df.iloc[:,:-1 ]
y = df.iloc[:,-1]

In [211]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, shuffle=False)

In [212]:
print(y_train.value_counts())
print(y_test.value_counts())

close_log_return_dir
0    13956
1    13916
Name: count, dtype: int64
close_log_return_dir
0    3503
1    3465
Name: count, dtype: int64


### Scikit-learn Logistic Regression

In [213]:
model = LogisticRegression(max_iter=200)

In [214]:
model.fit(X_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,200
,multi_class,'deprecated'


In [215]:
accuracy_score(y_train, model.predict(X_train))

0.5255453501722158

In [216]:
accuracy_score(y_test, model.predict(X_test))


0.5213834672789897

In [217]:
cm = confusion_matrix(y_test, model.predict(X_test))

<p style="color: yellow">TP, FP, TN, FN are purely relative to whatever you declare as the positive class.</p>

    - 1 is UP prediction
    - 0 is DOWN prediction
    - 1 is positive class
    - 0 is negative class



In [218]:
TN = cm[0][0]   #True Down  - When it should be down and model predicted down
FN = cm[1][0]   #False down - When it should be up but the model predicted down
FP = cm[0][1]   #False up   - When it should be down but the model predicted up
TP = cm[1][1]   #True UP    - When it should be up and the model predicted up

In [219]:
print(TN), print(TP), print(FN), print(FN)

1843
1790
1675
1675


(None, None, None, None)

#### Evaluate win rate

    Accuracy Measure

In [220]:
(TP+TN)/(TP+TN+FP+FN) 

np.float64(0.5213834672789897)

#### Evaluate the up Predictibility

In [221]:
TP/(TP+FP)

np.float64(0.518840579710145)

#### Evaluate the up Predictibility

<p style="color: yellow; font-size: 20px">Recall</p>
    


In [222]:
TP/(TP+FN)  # Here we are essentially looking at - from all the True classes, how many True classes I predicted correctly

np.float64(0.5165945165945166)

<p style="color: yellow; font-size: 20px">Precision</p>

Precision is, for all of my predictions for a particular class, how many times I was right


In [223]:
TP/(TP+FP) # From all the times I predicted True/UP, How many were actually true/up

np.float64(0.518840579710145)

#### Evaluate down predictability

<p style="color: yellow; font-size: 20px">Recall</p>


In [224]:
TN / (TN+FP) # Here we are essentially looking at - from all the False classes, how many false classes I predicted correctly

np.float64(0.5261204681701399)

In [225]:
TN / (TN+FN)    # From as many times I predicted false, how many were actually false.

np.float64(0.5238772029562251)

###### **PRECISION**
###### "When I predict UP, how often am I actually right?"

```python
Precision = TP / (TP + FP)
```

**Out of all my UP predictions, how many were actually correct?**

- **Focus:** My predicted UP events (my actions)
- **Fear:** False Positives (wrongly predicting UP when it's actually DOWN)
- **High precision = When I say UP, you can trust me — but I might be missing opportunities**

---

###### **RECALL**
###### "How many actual UP moves did I successfully catch?"

```python
Recall = TP / (TP + FN)
```

**Out of all the real UP days, how many did my model correctly predict as UP?**

- **Focus:** The actual UP events (reality)
- **Fear:** False Negatives (missing UP days)
- **High recall = I catch most of the UP moves, even if I sometimes make wrong predictions**

---

###### **The Key Difference**

| | Precision | Recall |
|---|-----------|--------|
| **Perspective** | YOUR predictions | REALITY's events |
| **Denominator** | What YOU predicted | What ACTUALLY happened |
| **Question** | "Am I accurate?" | "Am I thorough?" |
| **Trading analogy** | "Hit rate of my signals" | "% of opportunities captured" |

---

###### **Trading Strategy Examples**

###### **High Precision, Low Recall Strategy**
- Very selective, only trades slam-dunk setups
- Few trades, but most are winners
- Good for: High transaction costs, limited capital, risk-averse

###### **Low Precision, High Recall Strategy**
- Casts a wide net, takes many signals
- Catches most moves, but many false alarms
- Good for: Low transaction costs, diversification, systematic execution

###### **Balanced Strategy**
- Optimizes F1-score (harmonic mean of precision and recall)
- Trades off some accuracy for better coverage
- Good for: Most real-world trading applications

---

###### **The Trade-off**

You can't maximize both simultaneously:

- ↑ Threshold → ↑ Precision, ↓ Recall (be picky)
- ↓ Threshold → ↓ Precision, ↑ Recall (be aggressive)

**The optimal balance depends on your strategy's economics.**


#### Directional Balance : Short Ratio

In [226]:
short_ratio = (FN+TN)/(FN+TN+FP+FP)
short_ratio

np.float64(0.5144779175197426)

#### Directional Balance: Long Ratio

In [227]:
long_ratio = (FP+TP)/(FN+TN+FP+FP)
long_ratio

np.float64(0.5045334893243638)

#### Directional Imbalance

In [228]:
short_ratio/long_ratio

np.float64(1.0197101449275363)